In [2]:
%autoreload 2

# to make life easier, import all the needed stuff from workspace
# With autoreload enabled this also has the added benefit of making imported functionality
# available to the notebook without the need to rerun the importing cell.
from workspace import *

## Loading Model

In [3]:
from zipfile import ZipFile

In [8]:
with open("char-rnn-200.tar", 'rb') as src:
    main_loop = load(src)

In [20]:
main_loop.profile.total

defaultdict(int,
            {('after_training',): 0,
             ('after_training', 'Checkpoint'): 0,
             ('after_training', 'DataStreamMonitoring'): 4.118200013181195e-05,
             ('after_training', 'FinishAfter'): 2.182299976993818e-05,
             ('after_training', 'Printing'): 0.0005432170000858605,
             ('before_training',): 0.0001758740000070702,
             ('before_training', 'Checkpoint'): 2.6424000012070792e-05,
             ('before_training',
              'DataStreamMonitoring'): 4.230599999743845e-05,
             ('before_training', 'FinishAfter'): 1.5802999996594735e-05,
             ('before_training', 'Printing'): 5.913699999382516e-05,
             ('initialization',): 12.519453603999992,
             ('training',): 5399.65846744,
             ('training', 'after_epoch'): 328.7014877679999,
             ('training', 'after_epoch', 'Checkpoint'): 0.00019791700060523,
             ('training',
              'after_epoch',
              'DataS

## Playing with Main Loops

In [11]:
from word_embeddings import main_loop

In [3]:
main_loop.run()


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 test_simple_entropy_apply_cost: 5.717057105039682


-------------------------------------------------------------------------------
AFTER ANOTHER EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: False
	 epochs_done: 1
	 iterations_done: 23
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 23:
	 test_simple_entropy_apply_cost: 5.430238805431428
	 training_finish_requ